# HOMEWORK 6: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming)

We will focus only on the Object Classification task for this homework.

In this homework, you are asked compare different text classification models in terms of accuracy and inference time.

You will need to build 3 different models.

1. A model based on tf-idf
2. A model based on MUSE
3. A model based on wangchanBERTa

**You will be ask to submit 3 different files (.pdf from .ipynb) that does the 3 different models. Finally, answer the accuracy and runtime numbers in MCV.**

This homework is quite free form, and your answer may vary. We hope that the processing during the course of this assignment will make you think more about the design choices in text classification.

In [10]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2025-06-21 14:33:46--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6 [following]
--2025-06-21 14:33:46--  https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc97f2bc7a855bbd686718c4eed3.dl.dropboxusercontent.com/cd/0/inline/CsDWU9YhVNrQEWtsbEvcLVoMdZtRfJq3k4GUMVgAHeCMcQYv_XAjYZQi8iX0zEz3KmVH5I_zBgeNcM-rJ64O-3Mi8nYFc-FROjcdpFBUkjcavtocmRNsYvY1OnxBd2IxQS8/file# [following]
--2025-06-21 14:33:47--  https://uc97f2bc7a855bbd686718c4ee

In [11]:
!pip install pythainlp

## Import Libs

In [12]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score

## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [13]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [14]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 0.1:
You will have to remove unwanted label duplications as well as duplications in text inputs.
Also, you will have to trim out unwanted whitespaces from the text inputs.
This shouldn't be too hard, as you have already seen it in the demo.



In [15]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [16]:
# TODO.1: Data cleaning
data_df = data_df[["Sentence Utterance", "Object"]]
data_df.columns = ['input', 'raw_label']
display(data_df.describe())
display(data_df.raw_label.unique())

,input,raw_label
count,16175,16175
unique,13389,33
top,บริการอื่นๆ,service
freq,97,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

Split data into train, valdation, and test sets (normally the ratio will be 80:10:10 , respectively). We recommend to use train_test_spilt from scikit-learn to split the data into train, validation, test set.

In addition, it should split the data that distribution of the labels in train, validation, test set are similar. There is **stratify** option to handle this issue.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Make sure the same data splitting is used for all models.

In [17]:
data_df['clean_label']=data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)
display(data_df.describe())
display(data_df.clean_label.unique())

,input,clean_label
count,16175,16175
unique,13389,26
top,บริการอื่นๆ,service
freq,97,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [18]:
data_df = data_df.drop_duplicates("input", keep="first")
data_df = data_df.reset_index(drop=True)
display(data_df.describe())
data_df['clean_label'].value_counts()

,input,clean_label
count,13389,13389
unique,13389,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,service
freq,1,2111


,count
clean_label,
service,2111
package,1797
internet,1790
balance,1482
promotion,1145
suspend,730
payment,641
phone_issues,581
bill,540


In [19]:
data_df = data_df[data_df.clean_label != 'contact']

#Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:**

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:**

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:**

In [20]:
import attacut
import time
from tqdm import tqdm
import pythainlp

In [21]:
data = data_df.copy().to_numpy()

unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

display(num_2_label_map)
display(label_2_num_map)

data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])

{0: 'payment',
 1: 'package',
 2: 'suspend',
 3: 'internet',
 4: 'phone_issues',
 5: 'service',
 6: 'nontruemove',
 7: 'balance',
 8: 'detail',
 9: 'bill',
 10: 'credit',
 11: 'promotion',
 12: 'mobile_setting',
 13: 'iservice',
 14: 'roaming',
 15: 'truemoney',
 16: 'information',
 17: 'lost_stolen',
 18: 'balance_minutes',
 19: 'idd',
 20: 'garbage',
 21: 'ringtone',
 22: 'rate',
 23: 'loyalty_card',
 24: 'officer'}

{'payment': 0,
 'package': 1,
 'suspend': 2,
 'internet': 3,
 'phone_issues': 4,
 'service': 5,
 'nontruemove': 6,
 'balance': 7,
 'detail': 8,
 'bill': 9,
 'credit': 10,
 'promotion': 11,
 'mobile_setting': 12,
 'iservice': 13,
 'roaming': 14,
 'truemoney': 15,
 'information': 16,
 'lost_stolen': 17,
 'balance_minutes': 18,
 'idd': 19,
 'garbage': 20,
 'ringtone': 21,
 'rate': 22,
 'loyalty_card': 23,
 'officer': 24}

In [22]:
def strip_str(string):
    return string.strip()

data[:,0] = np.vectorize(strip_str)(data[:,0])

In [72]:
x = data[:, 0]
y = data[:, 1]

In [27]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [108]:
X_train, X_temp, y_train, y_temp = train_test_split(x, y, stratify=y, test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)

In [75]:
vocab = defaultdict(lambda: len(vocab))
vocab["<UNK>"] = 0

token_count = defaultdict(int)

for i in tqdm(range(len(X_train))):
    tokens = attacut.tokenize(X_train[i])
    for token in tokens:
        vocab[token]
        token_count[token] += 1

100%|██████████| 10708/10708 [02:04<00:00, 85.76it/s]


In [76]:
len(vocab)

4728

In [77]:
vocab_ = defaultdict(lambda: len(vocab_))
for i in tqdm(range(len(data))):
    tokens = attacut.tokenize(data[i][0])
    for token in tokens:
        vocab_[token]

100%|██████████| 13385/13385 [02:51<00:00, 78.21it/s]


In [100]:
print(len(vocab_))

5445


In [78]:
oov_count = 0
for i in tqdm(range(len(X_test))):
    tokens = attacut.tokenize(X_test[i])
    for token in tokens:
        token_count[token] += 1
        if token not in vocab:
            oov_count += 1

100%|██████████| 1339/1339 [00:15<00:00, 84.85it/s]


In [101]:
print(oov_count)

373


In [99]:
sorted_token_count = sorted(token_count.items(), key=lambda x: x[1], reverse=True)
sorted_token_count[:10]

[(' ', 22079),
 ('ค่ะ', 5487),
 ('จะ', 4885),
 ('ครับ', 4601),
 ('ได้', 3509),
 ('ไม่', 3311),
 ('ใช้', 3062),
 ('สอบถาม', 2877),
 ('ว่า', 2473),
 ('ผม', 2469)]

In [80]:
from pythainlp.corpus.common import thai_stopwords

stopwords = thai_stopwords()

In [81]:
def string_to_token_ids(X: np.ndarray):
    new_X = np.zeros(X.shape, dtype=object)
    for i in tqdm(range(len(X))):
        token_ids = []
        tokens = attacut.tokenize(X[i])
        tokens = [token for token in tokens if token not in stopwords]

        for token in tokens:
            if token in vocab:
                token_ids.append(vocab[token])
            else:
                token_ids.append(vocab["<UNK>"])

        new_X[i] = token_ids

    return new_X

In [109]:
X_train = string_to_token_ids(X_train)
X_test = string_to_token_ids(X_test)

100%|██████████| 1339/1339 [00:15<00:00, 86.51it/s]


In [110]:
num_docs_with_w = defaultdict(int)
for i in tqdm(range(len(X_train))):
    unique_tokens = set(X_train[i])

    for token in unique_tokens:
        num_docs_with_w[token] += 1
# sort token idx
num_docs_with_w = dict(sorted(num_docs_with_w.items(), key=lambda x: x[0]))

idf = np.log(len(X_train) / np.array(list(num_docs_with_w.values())))

100%|██████████| 10708/10708 [00:00<00:00, 521257.71it/s]


In [83]:
def gen_tf_idf_for_sample(sample):
    N = len(sample)
    tf = defaultdict(int)
    for token in sample:
        tf[token] += 1
    for token in tf:
        tf[token] = tf[token] / N
    tf_idf = np.zeros(len(vocab))
    for token in tf:
        if token < len(idf)-1:
            tf_idf[token] = tf[token] * idf[token]

    return tf_idf

def gen_tf_idf(X):
    new_X = np.zeros((X.shape[0], len(vocab)))
    for i, sample in enumerate(X):
        new_X[i] = gen_tf_idf_for_sample(sample)

    return new_X

In [111]:
X_train = gen_tf_idf(X_train)
X_test = gen_tf_idf(X_test)
y_train = y_train.copy().astype(int)
y_test = y_test.copy().astype(int)

In [112]:
lr = LogisticRegression(class_weight='balanced', max_iter=100, random_state=2025)
lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=2025)

In [113]:
predictions = lr.predict(X_test)

In [114]:
print("Model Acc. on test data %f%%"
       % ((y_test == predictions).sum() / y_test.shape[0] * 100))

Model Acc. on test data 60.119492%


# Model 2 MUSE

Build a simple logistic regression model using features from the MUSE model.

Which MUSE model will you use? Why?

**Ans:**

MUSE is typically used with tensorflow. However, there are some pytorch conversions made by some people.

https://huggingface.co/sentence-transformers/use-cmlm-multilingual
https://huggingface.co/dayyass/universal-sentence-encoder-multilingual-large-3-pytorch

In [116]:
from sentence_transformers import SentenceTransformer
from huggingface_hub import notebook_login

In [117]:
notebook_login()

In [118]:
sentences = data[:,0]
model = SentenceTransformer('sentence-transformers/use-cmlm-multilingual')
embeddings = model.encode(sentences, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/419 [00:00<?, ?it/s]

In [119]:
X = embeddings.copy()
y = data[:, 1]

In [120]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [121]:
lr = LogisticRegression(class_weight='balanced', max_iter=500, random_state=2025)
lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=2025)

In [122]:
print("Model Acc. on test data %f%%"
       % ((y_test == predictions).sum() / y_test.shape[0] * 100))

Model Acc. on test data 60.119492%


# Model 3 WangchanBERTa

We ask you to train a WangchanBERTa-based model.

We recommend you use the thaixtransformers fork (which we used in the PoS homework).
https://github.com/PyThaiNLP/thaixtransformers

The structure of the code will be very similar to the PoS homework. You will also find the huggingface [tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) useful. Or you can also add a softmax layer by yourself just like in the previous homework.

Which WangchanBERTa model will you use? Why? (Don't forget to clean your text accordingly).

**Ans:**


In [2]:
!pip install -q  transformers==4.30.1 datasets evaluate thaixtransformers
!pip install peft==0.10.0

In [6]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 47.9 MB/s eta 0:00:00


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, DataCollatorWithPadding, AutoModel
from tqdm import tqdm
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from torchmetrics import Accuracy

from thaixtransformers import Tokenizer
from torch import nn
import torch.nn.functional as F

In [8]:
tokenizer = Tokenizer("airesearch/wangchanberta-base-att-spm-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'WangchanbertaTokenizer'.


In [23]:
for i, sample in tqdm(enumerate(data)):
    cleaned_sentence = sample[0].replace('ํา', "ำ")
    data[i, 0] = tokenizer(cleaned_sentence)

13385it [00:05, 2328.93it/s]


In [24]:
X = data[:, 0]
y = data[:, 1]

In [28]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)

In [29]:
class CallCenterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(val) for key, val in self.encodings[idx].items()
            if key in ['input_ids', 'attention_mask'] # take only input_ids and attention_mask fields
        }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [30]:
train_dataset = CallCenterDataset(X_train, y_train)
val_dataset = CallCenterDataset(X_val, y_val)
test_dataset = CallCenterDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

In [31]:
class BaseModel(LightningModule):
    def __init__(
          self,
          model_name: str = 'airesearch/wangchanberta-base-att-spm-uncased',
          learning_rate: float = 2e-5
    ):
        super().__init__()
        self.save_hyperparameters()

        self.encoder = AutoModel.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def get_embeddings(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids, attention_mask)
        hidden_states = outputs.last_hidden_state

        # [CLS] token is the first token in the sequence (index 0)
        cls_embeddings = hidden_states[:, 0, :]  # [batch_size, hidden_size]

        return cls_embeddings

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

    def forward(self, input_ids, attention_mask):
        return self.get_embeddings(input_ids, attention_mask)

In [32]:
class LMWithLinearClassfier(BaseModel):
    def __init__(
          self,
          model_name: str = 'airesearch/wangchanberta-base-att-spm-uncased',
          ckpt_path: str = None,
          learning_rate: float = 2e-5,
          freeze_encoder_weights: bool = False
    ):
        super().__init__(
            model_name,
            learning_rate
        )
        self.save_hyperparameters()

        if ckpt_path:
            checkpoint = torch.load(ckpt_path)
            encoder_state_dict = {k.replace("encoder.", ""): v for k, v in checkpoint["state_dict"].items() if k.startswith("encoder.")}
            self.encoder.load_state_dict(encoder_state_dict)

        self.linear_layer = nn.Linear(768, 25)

        if freeze_encoder_weights:
          self.freeze_weights(self.encoder)  # Freeze model

        self.accuracy = Accuracy(task='multiclass', num_classes=25)

    def freeze_weights(self, model):
        for param in model.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        cls_embeddings = self.get_embeddings(input_ids, attention_mask)
        logits = self.linear_layer(cls_embeddings)
        return logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        logits = self.forward(input_ids, attention_mask)

        loss = F.cross_entropy(logits, labels)
        self.log('train_loss', loss, prog_bar=True)

        acc = self.accuracy(logits, labels)
        self.log('train_acc', acc, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        logits = self.forward(input_ids, attention_mask)

        loss = F.cross_entropy(logits, labels)
        self.log('val_loss', loss, prog_bar=True)

        acc = self.accuracy(logits, labels)
        self.log('val_acc', acc, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        logits = self.forward(input_ids, attention_mask)

        loss = F.cross_entropy(logits, labels)
        self.log('test_loss', loss, prog_bar=True)

        acc = self.accuracy(logits, labels)
        self.log('test_acc', acc, prog_bar=True)

        return loss

In [33]:
model = LMWithLinearClassfier(
    'airesearch/wangchanberta-base-att-spm-uncased',
    ckpt_path=None,
    freeze_encoder_weights=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_acc",  # Metric to monitor
    mode="max",  # "min" for loss, "max" for accuracy
    save_top_k=1,  # Save only the best model(s)
    save_weights_only=True, # Saves only weights, not the entire model
    dirpath="./checkpoints/", # Path where the checkpoints will be saved
    filename="best_pretrained_w_linear_model-{epoch}-{val_acc:.2f}", # Customized name for the checkpoint
    verbose=True,
)

linear_trainer = Trainer(
    max_epochs=5,
    accelerator='auto',
    callbacks=[checkpoint_callback], # Add the ModelCheckpoint callback
    gradient_clip_val=1.0,
    precision=16, # Mixed precision training
    devices=1,
)

/usr/local/lib/python3.11/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [35]:
start_time = time.time()
# Train the model
linear_trainer.fit(model, train_loader, val_loader)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type               | Params | Mode 
------------------------------------------------------------
0 | encoder      | CamembertModel     | 105 M  | eval 
1 | linear_layer | Linear             | 19.2 K | train
2 | accuracy     | MulticlassAccuracy | 0      | train
------------------------------------------------------------
105 M     Trainable params
0         Non-trainable params
105 M     Total params
421.055   Total estimated model params size (MB)
2         Modules in train mode
228       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 335: 'val_acc' reached 0.68460 (best 0.68460), saving model to '/content/checkpoints/best_pretrained_w_linear_model-epoch=0-val_acc=0.68.ckpt' as top 1


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 1340: 'val_acc' reached 0.76457 (best 0.76457), saving model to '/content/checkpoints/best_pretrained_w_linear_model-epoch=3-val_acc=0.76.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 1675: 'val_acc' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Total time: 219.17546963691711


In [36]:
start_time = time.time()
result = linear_trainer.test(model, test_loader)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7453323602676392     │
│         test_loss         │    1.0605791807174683     │
└───────────────────────────┴───────────────────────────┘

Total time: 1.54746675491333


After you

# Comparison

After you have completed the 3 models, compare the accuracy, ease of implementation, and inference speed (from cleaning, tokenization, till model compute) between the three models in mycourseville.